In [1]:
import nltk, string
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import word_tokenize

## Task 1 - Corpus Analysis

For Task 1, the goal is to simply identify the top 100 most frequent tokens within the corpus. The idea is to use this data to better inform the construction of the inverted index. Since the corpus contains not only the text files containing data for a given Simpsons episode but also some CSV files, my first step is to separate the text files from the corpus using a PlaintextCorpusReader object from nltk, passing it the file pattern ‘.*.txt’ i.e. extracting file names which are comprised of any number of characters followed by the text file extension. This ensures CSV files are not considered in this analysis seeing as how the focus is on the data relating to the episodes.

Once these text files have been extracted, I then tokenize these text files, create a list of all tokens present in the corpus and determine the most common ones. The process of extracting these 100 most frequent terms is made simple by the nltk library, being able to simply create a frequency distribution of the tokens in the corpus by constructing a FreqDist nltk object after compiling the tokens across the various documents into one list, then extract the 100 most common words using its
‘most_common’ method and display them i.e. using FreqDist.most_common(100).

However, the main point of consideration for this task is the pre-processing necessary in order to ensure that the tokens depicted as most common would actually convey meaning relative to the context of the corpus. I chose to apply three pre-processing steps to each document before compiling the tokens from them; Case folding, Stop Word
Removal and Punctuation Removal. Functions for each step were created and are called by the preprocess_task1 method (after they have been tokenized using the words(docid) method of the PlaintextCorpusReader object. This takes the doc id of the document in the corpus one wishes to tokenize and returns a list of all the tokens the nltk tokenizer picked up).

In [4]:
# Reading the relevant files
corpus_root = r"C:\Users\halor\Desktop\NLP_DocumentIndex\Simpsons"
file_pattern = r".*.txt"

wordlists = PlaintextCorpusReader(corpus_root, file_pattern)
fileids = wordlists.fileids()

In [5]:
# Casefolding by lowercasing all words in document
def casefold(words):
    words_lower = [word.lower() for word in words]
    return words_lower
    
# Removing all english stopwords using nltk stopword dictionary
def remove_stop_words(words):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    content = [w for w in words if w not in stopwords]
    return content

# Removing all punctuation i.e. . , ' " 
def remove_punctuation(words):
    punc_table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(punc_table) for w in words]
    
    # Above lines replace punctuation tokens with whitespace, so must
    # filter them out of the wordlist
    while("" in stripped):
        stripped.remove("")
    
    return stripped

In [6]:
# Function for pre-processing. Calls relevant functions in sequence
def preprocess_task1(words):
    words_casefolded = casefold(words)
    words_nostops = remove_stop_words(words_casefolded)
    words_nopunc = remove_punctuation(words_nostops)
    return words_nopunc

In [7]:
# Retrieving the top 100 most frequent words

# Preprocessing each file and merging the result into a unified list
all_words_in_corpus = []
for f_id in fileids:
    wordlist_processed = preprocess_task1(wordlists.words(f_id))
    all_words_in_corpus.extend(wordlist_processed)
    
# Next creating a frequency distribution of the results, and retrieving the most common words
fd = nltk.FreqDist(all_words_in_corpus)
for rank, fd_for_word in enumerate(fd.most_common(100)): 
    print("RANK: {} TOKEN: {} FREQ: {}".format(rank + 1, fd_for_word[0], fd_for_word[1]))

RANK: 1 TOKEN: episode FREQ: 1341
RANK: 2 TOKEN: homer FREQ: 838
RANK: 3 TOKEN: simpsons FREQ: 549
RANK: 4 TOKEN: bart FREQ: 517
RANK: 5 TOKEN: show FREQ: 404
RANK: 6 TOKEN: lisa FREQ: 388
RANK: 7 TOKEN: marge FREQ: 310
RANK: 8 TOKEN: season FREQ: 284
RANK: 9 TOKEN: 4 FREQ: 279
RANK: 10 TOKEN: 1 FREQ: 260
RANK: 11 TOKEN: one FREQ: 258
RANK: 12 TOKEN: 2 FREQ: 253
RANK: 13 TOKEN: 3 FREQ: 250
RANK: 14 TOKEN: 5 FREQ: 224
RANK: 15 TOKEN: film FREQ: 192
RANK: 16 TOKEN: 6 FREQ: 190
RANK: 17 TOKEN: episodes FREQ: 189
RANK: 18 TOKEN: edit FREQ: 188
RANK: 19 TOKEN: references FREQ: 187
RANK: 20 TOKEN: first FREQ: 172
RANK: 21 TOKEN: scene FREQ: 171
RANK: 22 TOKEN: guest FREQ: 169
RANK: 23 TOKEN: production FREQ: 163
RANK: 24 TOKEN: krusty FREQ: 161
RANK: 25 TOKEN: fox FREQ: 157
RANK: 26 TOKEN: written FREQ: 155
RANK: 27 TOKEN: television FREQ: 148
RANK: 28 TOKEN: burns FREQ: 144
RANK: 29 TOKEN: jean FREQ: 141
RANK: 30 TOKEN: family FREQ: 141
RANK: 31 TOKEN: reference FREQ: 141
RANK: 32 TOKEN: pl

## Task 2.1 - Inverted Index

For Task 2, the goal is to construct a data structure – the Inverted Index – that would contain information regarding the presence of a given word from the corpus in a document as well as its relative position within the document. Next was to include functionality to allow the search of user entered queries within the corpus.

### Index Creation and Structure – relevant method(s): index_corpus

I opted to create a unigram index. This was done to prevent the dimensionality explosion which can occur with an ngram index – there would be a vast number of keys within the inverted index (since every possible n-gram would need to be a key). This would increase the size of the data structure and cause the memory requirements for the index to scale poorly with the size of the corpus. While this increases the complexity of finding multi-word terms, this can be circumnavigated by intelligent traversal of the unigram index when searching for terms containing multiple tokens.

An example showing the structure of my index is depicted below:

{
    token1 : {
        docid1 : [0, 5, 6],
        docid2 : [1, 3, 9]
    },
    token2 : {
        docid1 : [1, 3, 9],
        docid2 : [9, 15, 23]
    }
}

The index is implemented as a python dictionary, where the key is a token in the corpus (after pre-processing each document and extracting the tokens remaining alongside all the documents they belong too) and the value is another dictionary for the token in question. This nested dictionary has keys which are made up of the id’s of the docs the token appears in in the corpus, and the value in the dictionary for each of these docid’s is a list of indexes depicting the position of the token in the document after pre-processing/tokenization. For example, based on the depiction above, token1 can be found in positions 0, 5 and 6 in the document under docid1 after it was tokenized and pre-processed.


### Pre-processing – relevant method(s): process_document
I followed the same pre-processing steps and used the same implementation functions as I did for Task 1.

### Searching – relevant method(s): proximity_search, perform_search, find_term
As previously mentioned, I created a unigram index, which I use to find single and multi-word search terms within the corpus. This search works by essentially performing a depth first search through the index to complete a given term. Each token is treated as a node in a graph and each of the document/position combinations is treated as an edge. For single word search terms, the index is simply queried, but for multi-word terms the index is traversed to construct the term and thus find its occurrences, with the window size being used to dictate whether a token should be included in the DFS or not. The obvious benefit to this approach is that it allowed searching for terms with more than one word to be possible even with a unigram index, however this would mean that for particularly long queries my algorithm would be less efficient than simply using an n-gram index. The output of this algorithm is a list of tuples in the form (docid, term_start, term_end), where docid is the document id the term was found in, term_start/term_end is the position of the start/end of the term in the tokenized, pre-processed document.

One thing to note is that for searching to work, the same pre-processing steps used on the documents when constructing the index had to be used on the query to ensure the index could be traversed accurately using it – if tokens were present in the query that weren’t in the index, many searches would fail unnecessarily.

In [8]:
class InvertedIndex:
    """
    Construct Inverted Index
    """
    def __init__(self, window):
        self.window = window
        self.corpus_indexed = None
        self.fileids = None
        
    def read_data(self, path: str) -> list:
        """
        Read files from a directory and then append the data of each file into a list.
        """
        wordlists = PlaintextCorpusReader(path, file_pattern)
        self.fileids = wordlists.fileids()
        data = []
        
        # Constructing list of tuples in the form (docid, raw doc contents)
        for f_id in self.fileids:
            data.append((f_id, wordlists.raw(f_id)))
        
        return data

    def process_document(self, document: str) -> list:
        """
        pre-process a document and return a list of its terms
        str->list"""
        
        # First, tokenize the document
        doc_tokenized = word_tokenize(document)
        
        # Then, casefolding the document by lowercasing everything. Making use of function created in Task 1
        document_lower = casefold(doc_tokenized)
        
        # Next, removing stop words, again using a function created in Task 1
        document_nostop = remove_stop_words(document_lower)
        
        # In addition, removing punctuation using func from Task 1
        document_nopunc = remove_punctuation(document_nostop)
        
        return document_nopunc
    
    def index_corpus(self, documents: list) -> None:
        """
        index given documents
        list->None"""
        
        # Index created in the form of a python dictionary:
        # { 
        #   token1 : {
        #            docid1 : [0, 5, 6],
        #            docid2 : [1, 3, 9]
        #           },
        #   token2 : {
        #            docid1 : [1, 3, 9],
        #            docid2 : [9, 15, 23]
        #           }
        # }
        # i.e. The keys of the dictionary are each unique token present in the corpus after pre-processing, and the value in
        # the dictionary for each token is another dictionary, where the keys are the docids of the documents the token can be 
        # found in, and the value these docids map to is a list of numbers denoting the position of the token in the document 
        # after the document was pre-processed. Thus in the example given above, token1 is present in docid1 in positions 0, 5
        # and 6 after the document was processed into a list of tokens e.g. docid1 -> [token1, .., .., .., .., token1, token1]
        corpus_indexed = {}
        
        for doc in documents:
            docid = doc[0]
            doc_processed = self.process_document(doc[1])
            for i in range(len(doc_processed)):
                token = doc_processed[i]
                if (token not in corpus_indexed):
                    corpus_indexed[token] = { docid : [i] }
                else:
                    token_index = corpus_indexed[token]
                    if (docid not in token_index):
                        token_index[docid] = [i]
                    else:
                        token_index[docid].append(i)
        
        self.corpus_indexed = corpus_indexed   
        
    # Finds all the occurences of a term using the corpus index, based on window size. Returns 
    # a list of tuples of the form (docid, term_start_index, term_end_index) for every occurence of
    # the term
    def find_term(self, term: list, docid: str, current_word_index: int, next_word_index: int, term_start: int, 
                  term_end: int) -> list:
        """
        find occurrences of the given term in the doc under docid
        list->list"""
        term_occurrences = []
        
        # First things first - we have to check if the length of the term (post processing) is > 1 as if it isn't,
        # we simply need to return the places the term occurs in the document - something easily retrieve from the
        # words entry in the index
        if (len(term) == 1):
            word = term[0]
            
            if word not in self.corpus_indexed.keys():
                return term_occurrences
            
            term_ii_entry = self.corpus_indexed[word]
            
            if docid not in term_ii_entry.keys():
                return term_occurrences
            
            for pos in term_ii_entry[docid]:
                term_occurrences.append((docid, pos, pos))
            
            return term_occurrences
            
        # Check if the next_word_index has exceeded the length of the term - this indicates that we have processed
        # the entire term. If it has, we should construct the relevant tuple
        if next_word_index >= len(term):
            return [(docid, term_start, term_end)]
        
        # Initialising relevant variables. current_word is the word currently being used as the base of the term,
        # while next_word is the word that comes directly after current_word
        current_word = term[current_word_index]
        next_word = term[next_word_index]
        
        # We attempt to find the occurrences of the term in the document under docid, starting with current_word
        
        # Should check for each words existence in the corpus first
        if current_word not in self.corpus_indexed.keys() or next_word not in self.corpus_indexed.keys():
            return term_occurrences
        
        # First, retrieve the II entry for the current word
        current_word_ii_entry = self.corpus_indexed[current_word]
        
        # Then do the same for next_word (the word following the current word in the term)
        next_word_ii_entry = self.corpus_indexed[next_word]
        
        # Must ensure that both words appear in the same doc
        if docid not in current_word_ii_entry.keys() or docid not in next_word_ii_entry.keys():
            return term_occurrences
        
        # Then, using the positional information for the word in the doc, attempt to find the entire term
        current_word_positions = current_word_ii_entry[docid]
        next_word_positions = next_word_ii_entry[docid]
        
        for c_pos in current_word_positions:
            for n_pos in next_word_positions:
                # Establish the maximum position that the next word in the term can be in
                # in the document
                max_next_pos = c_pos + self.window
                if n_pos > c_pos and n_pos <= max_next_pos:
                    # If the start of the term has not been set yet, it is passed to subsequent recursive calls
                    if term_start == -1:
                        curr_result = self.find_term(term=term, current_word_index=current_word_index + 1, 
                                                next_word_index=next_word_index + 1, docid=docid,
                                                term_start=c_pos, term_end=n_pos)
                        term_occurrences.extend(curr_result)
                    else:
                        curr_result = self.find_term(term=term, current_word_index=current_word_index + 1, 
                                                next_word_index=next_word_index + 1, docid=docid,
                                                term_start=term_start, term_end=n_pos)
                        term_occurrences.extend(curr_result)
                            
        return term_occurrences
    
    # Search for occurences of the given search term within each document and compile the results
    def perform_search(self, term_processed: list) -> list:
        """
        find occurrences of the given term in the corpus
        list->list"""
        occurrences = []
        for fileid in self.fileids:
            res = self.find_term(term=term_processed, docid=fileid, current_word_index=0, next_word_index=1, term_start=-1,
                            term_end=-1)
            occurrences.extend(res)
        
        return occurrences
    
    # Construct the required dict return value for the proximity search
    # Takes the list of occurrences for the term
    def construct_proximity_search_result(self, term_occurrences: list) -> dict:
        """
        find occurrences of the given term in the doc under docid
        list->list"""
        result = {}
        if len(term_occurrences) == 0:
            return result
        
        for occurrence in term_occurrences:
            occurrence_docid = occurrence[0]
                
            if occurrence_docid in result.keys():
                result[occurrence_docid] += 1
            else:
                result[occurrence_docid] = 1
        
        return result
    
    def proximity_search(self, term1: str, term2: str) -> dict:
        """
        1) check whether given two terms appear within a window
        2) calculate the number of their co-existance in a document
        3) add the document id and the number of matches into a dict
        return the dict"""
        # Return empty dictionary if corpus has not yet been indexed
        if (self.corpus_indexed == None):
            return {}
        
        if (len(term1) == 0):
            # Process the term. This is needed to ensure that search terms can be compared
            # to the processed corpus
            term_processed = self.process_document(term2)
            
            # Then iterate over every document and compile occurences of the term
            occurrences = self.perform_search(term_processed)
            
            # process occurrences list and give result
            return self.construct_proximity_search_result(occurrences)
        elif (len(term2) == 0):
            # Process the term. This is needed to ensure that search terms can be compared
            # to the processed corpus
            term_processed = self.process_document(term1)
            
            # Then iterate over every document and compile occurences of the term
            occurrences = self.perform_search(term_processed)
            
            # process occurrences list and give result
            return self.construct_proximity_search_result(occurrences)
        else:
            # Process the terms. This is needed to ensure that search terms can be compared
            # to the processed corpus
            term1_processed = self.process_document(term1)
            term2_processed = self.process_document(term2)
            
            occurrences_term1 = self.perform_search(term1_processed)
            occurrences_term2 = self.perform_search(term2_processed)
            
            # process the two lists pairwise to give result
            
            # First check both terms were found
            if len(occurrences_term1) == 0 or len(occurrences_term2) == 0:
                return result
            
            # Then combine the results for each term. Here we are looking for occurrences of term1 followed by term2
            # within the window limit
            overlapping_occurrences = []
            for occurrence_term1 in occurrences_term1:
                for occurrence_term2 in occurrences_term2:
                    term1_docid = occurrence_term1[0]
                    term2_docid = occurrence_term2[0]
                    
                    term1_start = occurrence_term1[1]
                    term1_end = occurrence_term1[2]
                    
                    term2_start = occurrence_term2[1]
                    term2_end = occurrence_term2[2]
                    
                    max_term2_start = term1_end + self.window
                    
                    if term1_docid == term2_docid and term2_start >= term1_end and term2_start <= max_term2_start:
                        overlapping_occurrences.append((term1_docid, term1_start, term2_end))
            
            # Return the result
            return self.construct_proximity_search_result(overlapping_occurrences)

## Task 2.2 - Demo

Here one can experiment with the Inverted Index by experimenting with different search terms. I added a delimiter option to user entered queries to allow for multi-word search terms to be used. The item before the delimiter is treated as search term one, and the other as search term two – no matter how many tokens each contain. The results of the search for the given terms are returned in the form of a dictionary where the key is the docid the term(s) were found in and the value is the number of occurrences in that document. This is parsed to generate the outputs.

Some good test queries would be:
- homer
- homer simpson
- marge | homer
- homer nightmare | grave digger

Results can be confirmed by using CTRL-F to search the returned document for the query terms.

In [9]:
def display_result(search_result: dict, window_size: int) -> None:
    res_docids = search_result.keys()
    if len(res_docids) == 0:
        print("Term(s) not found")
    else:
        print("Window Size: {}".format(window_size))
        for doc in res_docids:
            print("DOCID: {} NUM_OCCURRENCES: {}".format(doc, search_result[doc]))

def main():
    "main call function"
    # The size of the window is set using the constructor
    window_size = 3
    index = InvertedIndex(window=window_size) # initilaise the index
    
    # specify the directory path in which files are located
    corpus = index.read_data(r"C:\Users\halor\Desktop\NLP_DocumentIndex\Simpsons")
    index.index_corpus(corpus) # index documents/corpus
    
    # insert a query. Each term should be separated with a | i.e. Homer Simpson | Marge Simpson -> term1 = Homer Simpson,
    # term2 = Marge Simpson
    search_term = input("Enter your query (separate each distinct term with a |): ")
    term_list = search_term.split("|")
    # write a demo to check entered search terms against the inverted index
        # 1) len(search _term) == one --> return the following: 
            # a) the number of documents in which a term appears.
            # b) all document ids in which a term appears.
    if len(term_list) == 1:
        search_result = index.proximity_search(search_term, "")
        print(index.corpus_indexed)
#         display_result(search_result, window_size)
    # 2) len(search_term) == 2 --> return the following: 
            # a) the number of documents in which the entered terms appear within a pre-defined window.
            # b) all document ids in which the terms appear within that window.
    elif len(term_list) == 2:
        search_result = index.proximity_search(term_list[0], term_list[1])
        display_result(search_result, window_size)
        
    return index
    
index = main()

Window Size: 3
DOCID: 3.7.txt NUM_OCCURRENCES: 1
